In [ ]:
import finis
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

### Question 2

The matrix is given by
$$
v_h^T \left(U^T \,W \,U + DUX^T \,W\, DUX + DUY^T \,W\, DUY\right) u_h = v_h^T \,A \,u_h
$$

### Question 3

The right hand side is given by
$$
v_h^T \, U^T \, W \, F_i = v_h^T \, F
$$
where $F_i$ is the right hand side function evaluated at the **i**ntegration points.

### Question 4

Solve the von Neumann model problem given by
$$ u - \Delta u = f \qquad \text{on} \ \Omega \\ \frac{\partial u}{\partial n} = 0 \qquad \text{on} \ \partial\Omega $$
on the grid $\Omega = (0, 2\pi)^2$ with the right hand side $f = \cos(x) \cos(y)$.

The analytic solution is given by $u = \frac13 \cos(x) \cos(y)$.

In [ ]:
%%time
vertices = np.array([[0.0, 0], [2*np.pi, 0], [2*np.pi, 2*np.pi], [0, 2*np.pi]])
mesh = finis.triangulate(vertices=vertices, max_area=1)
fe = finis.fe_space(mesh, order=2)

In [ ]:
finis.plot_mesh(mesh, vertex_numbers=False)

In [ ]:
%%time
u = lambda x,y: np.cos(x)*np.cos(y)/3.0
dux = lambda x,y: -np.sin(x)*np.cos(y)/3.0
duy = lambda x,y: -np.cos(x)*np.sin(y)/3.0

f = lambda x,y: np.cos(x)*np.cos(y)
F_i = f(fe['integ'][:,0], fe['integ'][:,1])
W = sp.spdiags(fe['w'], [0], m=fe['w'].size, n=fe['w'].size)
A = fe['U'].transpose().dot(W).dot(fe['U']) + \
    fe['DUX'].transpose().dot(W).dot(fe['DUX']) + \
    fe['DUY'].transpose().dot(W).dot(fe['DUY'])
F = fe['U'].transpose().dot(W).dot(F_i)
u_h = sp.linalg.spsolve(A, F)
u_ana = u(fe['dof'][:,0], fe['dof'][:,1])

print("Inf error = {}".format(np.amax(np.abs(u_h - u_ana))))

### Question 5

In [ ]:
fig = plt.figure(figsize=(9,5))
ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.plot_trisurf(fe['dof'][:,0], fe['dof'][:,1], u_h, linewidth=0.2, antialiased=True)
ax.set_title("Numeric Solution")

ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.plot_trisurf(fe['dof'][:,0], fe['dof'][:,1], u_ana, linewidth=0.2, antialiased=True)
ax.set_title("Analytic Solution")
plt.show()

### Question 6

In [ ]:
# L_2 error
xi = fe['integ'][:,0]
yi = fe['integ'][:,1]
integrand = (fe['U'].dot(u_h) - u(xi, yi))**2
L2 = np.sqrt(np.sum(fe['w'] * integrand))

# H_1 error
integrand = (fe['DUX'].dot(u_h) - dux(xi, yi))**2 + (fe['DUY'].dot(u_h) - duy(xi, yi))**2
H1 = np.sqrt(L2**2 + np.sum(fe['w'] * integrand))

print("L2 = {}\nH1 = {}".format(L2, H1))

In [ ]:
maxArea = np.logspace(0, 2, 20, base=0.1)
L2 = np.zeros(maxArea.shape)
H1 = np.zeros(maxArea.shape)
hs = np.zeros(maxArea.shape)

for i in range(maxArea.size):
    mesh = finis.triangulate(vertices=vertices, max_area=maxArea[i])
    fe = finis.fe_space(mesh, order=1, return_h=True)
   
    F_i = f(fe['integ'][:,0], fe['integ'][:,1])
    W = sp.spdiags(fe['w'], [0], m=fe['w'].size, n=fe['w'].size)
    A = fe['U'].transpose().dot(W).dot(fe['U']) + \
        fe['DUX'].transpose().dot(W).dot(fe['DUX']) + \
        fe['DUY'].transpose().dot(W).dot(fe['DUY'])
    F = fe['U'].transpose().dot(W).dot(F_i)
    u_h = sp.linalg.spsolve(A, F)
    u_ana = u(fe['dof'][:,0], fe['dof'][:,1])

    
    xi = fe['integ'][:,0]
    yi = fe['integ'][:,1]
    integrand = (fe['U'].dot(u_h) - u(xi, yi))**2
    L2[i] = np.sqrt(np.sum(fe['w'] * integrand))

    # H_1 error
    integrand = (fe['DUX'].dot(u_h) - dux(xi, yi))**2 + (fe['DUY'].dot(u_h) - duy(xi, yi))**2
    H1[i] = np.sqrt(L2[i]**2 + np.sum(fe['w'] * integrand))
    hs[i] = np.mean(fe['h'])

In [ ]:
plt.figure()
plt.loglog(hs, L2, '+-')
plt.loglog(np.array([hs[0], hs[-1]]), np.array([L2[0], L2[-1]]), 'r:')
plt.title("L2 error")
plt.show()

i = -6
print("Order = {}".format((np.log(L2[i])-np.log(L2[-1])) / (np.log(hs[i])-np.log(hs[-1]))))

In [ ]:
plt.figure()
plt.loglog(hs, H1, '+-')
plt.loglog(np.array([hs[0], hs[-1]]), np.array([H1[0], H1[-1]]), 'r:')
plt.title("H1 error")
plt.show()

i = -6
print("Order = {}".format((np.log(H1[i])-np.log(H1[-1])) / (np.log(hs[i])-np.log(hs[-1]))))

### Question 8

In [ ]:
markers = np.copy(mesh['vertex_markers']).squeeze()
inner_to_all = np.where(markers==0)[0]
#all_to_inner = -np.ones(shape=markers.shape, dtype=np.int32)
#all_to_inner[inner_to_all] = np.arange(inner_to_all.size)

row = inner_to_all
col = np.arange(inner_to_all.size)
data = np.ones((inner_to_all.size, ), dtype=np.float32)
P = sp.csr_matrix((data, (row, col)), shape=(markers.size, inner_to_all.size))
assert np.array_equal(np.squeeze(np.asarray(P.sum(axis=1), dtype=np.int32)), 1-markers)

In [ ]:
%%time
u = lambda x,y: np.sin(x)*np.sin(y)/3.0
dux = lambda x,y: np.cos(x)*np.sin(y)/3.0
duy = lambda x,y: np.sin(x)*np.cos(y)/3.0

f = lambda x,y: np.sin(x)*np.sin(y)
F_i = f(fe['integ'][:,0], fe['integ'][:,1])
W = sp.spdiags(fe['w'], [0], m=fe['w'].size, n=fe['w'].size)
A = fe['U'].transpose().dot(W).dot(fe['U']) + \
    fe['DUX'].transpose().dot(W).dot(fe['DUX']) + \
    fe['DUY'].transpose().dot(W).dot(fe['DUY'])
F = fe['U'].transpose().dot(W).dot(F_i)

Ad = P.transpose().dot(A).dot(P)
Fd = P.transpose().dot(F)
u_h_d = sp.linalg.spsolve(Ad, Fd)
u_h = P.dot(u_h_d)
u_ana = u(fe['dof'][:,0], fe['dof'][:,1])

print("Inf error = {}".format(np.amax(np.abs(u_h - u_ana))))

In [ ]:
fig = plt.figure(figsize=(9,5))
ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.plot_trisurf(fe['dof'][:,0], fe['dof'][:,1], u_h, linewidth=0.2, antialiased=True)
ax.set_title("Numeric Solution")

ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.plot_trisurf(fe['dof'][:,0], fe['dof'][:,1], u_ana, linewidth=0.2, antialiased=True)
ax.set_title("Analytic Solution")
plt.show()

### Error Plots

In [ ]:
f = lambda x,y: np.cos(x)*np.cos(y)
u = lambda x,y: np.cos(x)*np.cos(y)/3.0
dux = lambda x,y: -np.sin(x)*np.cos(y)/3.0
duy = lambda x,y: -np.cos(x)*np.sin(y)/3.0

finis.error_plot(vertices, f, u, dux, duy, order=2, dirichlet=False)

In [ ]:
f = lambda x,y: np.sin(x)*np.sin(y)
u = lambda x,y: np.sin(x)*np.sin(y)/3.0
dux = lambda x,y: np.cos(x)*np.sin(y)/3.0
duy = lambda x,y: np.sin(x)*np.cos(y)/3.0

finis.error_plot(vertices, f, u, dux, duy, order=2, dirichlet=True)